<a href="https://colab.research.google.com/github/j0KZ/options-api-financial-datasets/blob/main/options_api_financial_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi there 👋 - [financialdatasets.ai](https://www.financialdatasets.ai) is a stock market API that provides:


*   income statements
*   balance sheets
*   cash flow statements
*   sec filings
*   stock prices
*   options data
*   insider transactions

This is a quick start guide on how to use the **Options Chain** API.

**Questions?**

The full API documentation is available [here](https://docs.financialdatasets.ai/api-reference/endpoint/options/chain).

You can also DM me on [twitter](https://www.x.com/virattt).

In [ ]:
import getpass
import os

# You can get an API key here https://financialdatasets.ai/
os.environ["FINANCIAL_DATASETS_API_KEY"] = getpass.getpass()

In [ ]:
import requests

# add your API key to the headers
headers = {
    "X-API-KEY": os.environ["FINANCIAL_DATASETS_API_KEY"]
}

# set your query params
ticker = 'NVDA'    # data for 'AAPL', 'MSFT', 'BRK.B', 'NVDA', 'TSLA', 'GOOGL is free
limit = 10         # number of contracts to return

# create the URL
url = (
    f'https://api.financialdatasets.ai/options/chain'
    f'?ticker={ticker}'
    f'&limit={limit}'
)

# make API request
response = requests.get(url, headers=headers)

# parse options_chain from the response
options_chain = response.json().get('options_chain')

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Create a DataFrame from the options data
df = pd.DataFrame(options_chain)

# Define column mapping for better display
column_mapping = {
    'ticker': 'Option Symbol',
    'type': 'Type',
    'strike_price': 'Strike Price',
    'expiration_date': 'Expiration',
    'volume': 'Volume',
    'open_interest': 'Open Interest',
    'bid_price': 'Bid',
    'bid_size': 'Bid Size',
    'ask_price': 'Ask',
    'ask_size': 'Ask Size',
    'last_trade_price': 'Last Price',
    'last_trade_size': 'Last Size',
    'last_trade_time': 'Last Trade Time'
}

# Rename columns
df = df.rename(columns=column_mapping)

# Function to format currency values
def format_currency(value):
    if isinstance(value, (int, float)):
        return f"${value:,.2f}"
    return value

# Function to format numeric values
def format_numeric(value):
    if isinstance(value, (int, float)):
        if abs(value) >= 1e6:
            return f"{value/1e6:.2f}M"
        elif abs(value) >= 1e3:
            return f"{value/1e3:.1f}K"
        else:
            return f"{value:,}"
    return value

# Format currency columns
currency_columns = ['Strike Price', 'Bid', 'Ask', 'Last Price']
for col in currency_columns:
    df[col] = df[col].apply(format_currency)

# Format numeric columns
numeric_columns = ['Volume', 'Open Interest', 'Bid Size', 'Ask Size', 'Last Size']
for col in numeric_columns:
    df[col] = df[col].apply(format_numeric)

# Format date/time column - fixing timezone issue
df['Last Trade Time'] = df['Last Trade Time'].apply(lambda x: x.split(' EDT')[0])
df['Last Trade Time'] = pd.to_datetime(df['Last Trade Time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Select and reorder columns for display
display_columns = [
    'Option Symbol', 'Type', 'Strike Price', 'Expiration',
    'Bid', 'Ask', 'Last Price', 'Volume', 'Open Interest',
    'Last Trade Time'
]
df = df[display_columns]

# Define CSS styles
styles = [
    dict(selector="th", props=[
        ("background-color", "#1e3d59"),
        ("color", "white"),
        ("font-weight", "bold"),
        ("padding", "12px"),
        ("text-align", "center")
    ]),
    dict(selector="td", props=[
        ("padding", "10px 15px"),
        ("text-align", "right")
    ]),
    dict(selector="", props=[
        ("border-collapse", "collapse"),
        ("font-family", "Arial, sans-serif"),
        ("width", "100%")
    ])
]

# Simplified styling approach
def color_currency_columns(s, props=''):
    currency_cols = ['Strike Price', 'Bid', 'Ask', 'Last Price']
    return ['background-color: #f0f7ff' if s.name in currency_cols else '' for v in s]

# Format the DataFrame
styled_df = df.style\
    .set_properties(**{
        'background-color': '#ffffff',
        'color': '#333333',
        'border-bottom': '1px solid #e0e0e0'
    })\
    .apply(color_currency_columns, axis=0)\
    .set_table_styles(styles)\
    .hide(axis="index")

# Display the styled DataFrame
display(HTML(styled_df.to_html(index=False)))

Option Symbol,Type,Strike Price,Expiration,Bid,Ask,Last Price,Volume,Open Interest,Last Trade Time
NVDA241101C00050000,Call,$50.00,2024-11-01,$90.30,$90.95,$91.35,100,203,2024-10-28 10:58:58
NVDA241101C00055000,Call,$55.00,2024-11-01,$85.25,$86.00,$85.57,8,219,2024-10-28 12:43:57
NVDA241101C00060000,Call,$60.00,2024-11-01,$80.15,$81.00,$81.13,2,2,2024-10-28 15:12:04
NVDA241101C00065000,Call,$65.00,2024-11-01,$75.25,$76.00,$76.22,1,20,2024-10-28 15:40:00
NVDA241101C00066000,Call,$66.00,2024-11-01,$74.15,$75.05,$75.40,2,28,2024-10-28 10:26:11
NVDA241101C00067000,Call,$67.00,2024-11-01,$73.20,$74.05,$68.17,1,1,2024-10-11 12:03:05
NVDA241101C00068000,Call,$68.00,2024-11-01,$72.05,$73.05,$67.36,1,10,2024-10-11 12:05:36
NVDA241101C00069000,Call,$69.00,2024-11-01,$71.20,$72.05,$74.85,10,11,2024-10-25 11:15:45
NVDA241101C00070000,Call,$70.00,2024-11-01,$70.25,$71.00,$71.00,3,108,2024-10-28 15:04:26
NVDA241101C00071000,Call,$71.00,2024-11-01,$69.25,$70.05,$67.65,1,2,2024-10-14 10:59:24
